# Figure Code (R)

### Installing libraries and packages for analysis

In [ ]:
library(Seurat)
library(SeuratData)
library(SeuratWrappers)
library(sctransform)
library(monocle3)
library(SingleCellExperiment)
library(zellkonverter) # for readH5AD()
library(ggplot2)
library(ComplexHeatmap)
library(zoo)
library(dplyr)
library(magrittr)
library(viridis)
library(patchwork)
library(ggpubr)
library(png)
library(loomR)
library(R.utils)
library(EnhancedVolcano)

### Figure 1

In [ ]:
# Figure 1a made with Biorender.com

# Figure 1b
png("alpha_umap.png", units="in", width=5, height=5, res=300)
DimPlot(alpha_new, reduction = 'X_umap', cols = c('#30DC40','#FC5454','gray'))
dev.off()

png("prkcd_umap.png", units="in", width=8, height=6, res=300)
prkcd_features <- FeaturePlot(alpha_new, reduction = "X_umap", features = "Prkcd") & NoAxes()
dev.off()

png("kcnq5_umap.png", units="in", width=8, height=6, res=300)
kcnq5_features <- FeaturePlot(alpha_new, reduction = "X_umap", features = "Kcnq5") & NoAxes()
dev.off()

# Figure 1c
png("alpha_pca.png", units="in", width=5, height=5, res=300)
DimPlot(alpha_new, reduction = 'pca', cols = c('#30DC40','#FC5454','gray'))
dev.off()

# Figure 1d
png("pc1_elbow.png", units="in", width=8, height=6, res=300)
ElbowPlot(alpha_new, ndims = 20, reduction = "pca") + labs(title = "Variance Explained by Principal Components")
dev.off()

# Figure 1e
png("pc1_jackstraw.png", units="in", width=8, height=6, res=300)
alphaJ <- JackStraw(alpha_new, dims = 20)
alphaJ <- ScoreJackStraw(alphaJ, dims = 1:20)
JackStrawPlot(alphaJ, dims = 1:20, reduction = "pca")
dev.off()

# Figure 1f
png("pc1_heatmap_100.png", units="in", width=8, height=6, res=300)
DimHeatmap(alpha_new, nfeatures = 100, reduction = "pca", fast=FALSE) +
  (theme(text = element_text(size = 7)))
dev.off()

cell_color <- c('#30DC40','#FC5454','gray')
cell_types <- c('sf','ff', 'Unknown (not shown)')
axis_df <- data_frame('cellname' = names(pc1_coord_scaled), 'embeddings' = pc1_coord_scaled, 'celltype' = Idents(alpha_new))
axis_df$transp <- ifelse(axis_df$celltype == 'Unknown', 0.1, 1)
axis_plot <- ggplot(axis_df, aes(x = pc1_coord_scaled, y = 0, col = celltype))
axis_plot <- axis_plot + labs(col="Celltype") + scale_color_manual(values = cell_color, labels = cell_types, drop = FALSE) + theme_classic()
axis_plot <- axis_plot + geom_point(aes(y = 0), alpha = axis_df$transp) + scale_y_continuous(expand = c(0, 0), limits = c(0, 15)) + coord_cartesian(clip = 'off')
axis_plot <- axis_plot + theme(axis.line.y=element_blank(),
                               axis.text.y=element_blank(),
                               axis.title.x=element_blank(),
                               axis.title.y=element_blank(),
                               axis.ticks.y=element_blank(),
                               legend.position = "bottom")
png("1D_celltype.png", units="in", width=10, height=8, res=300)
axis_plot
dev.off()

pseudotime_plot <- ggplot(axis_df, aes(x = pc1_coord_scaled, y = 0, col = embeddings))
pseudotime_plot <- pseudotime_plot + labs(col="Fast Score") + scale_color_viridis_c(option = "plasma") + theme_classic()
pseudotime_plot <- pseudotime_plot + geom_point(aes(y = 0), alpha = axis_df$transp) + scale_y_continuous(expand = c(0, 0), limits = c(0, 15)) + coord_cartesian(clip = 'off')
pseudotime_plot <- pseudotime_plot + theme(axis.line.y=element_blank(),
                                           axis.text.y=element_blank(),
                                           axis.title.x=element_blank(),
                                           axis.title.y=element_blank(),
                                           axis.ticks.y=element_blank(),
                                           legend.position = "bottom")
png("1D_pseudotime.png", units="in", width=10, height=8, res=300)
pseudotime_plot
dev.off()

# Figure 1g
mnHist <- ggplot(axis_df, aes(x=embeddings, fill=celltype)) +
  geom_histogram(binwidth=0.1, boundary=0, alpha=rep(c(0.75, 0.35), c(20, 10)), position="identity") +
  xlab("Fast score") +
  ylab("Cell count") +
  # ggtitle("Motor Neuron Distributions along PC1") +
  scale_fill_manual(values = c('#30DC40','#FC5454','gray')) +
  theme(legend.position="top", plot.title = element_text(hjust = 0.5))

mnHistSep <- ggplot(axis_df, aes(x=embeddings, fill=celltype)) +
  geom_histogram(binwidth=0.1, boundary=0, alpha=0.75, position="identity") +
  xlab("Fast score") +
  ylab("Cell count") +
  # ggtitle("Motor Neuron Distributions along PC1") +
  scale_fill_manual(values = c('#30DC40','#FC5454','gray')) +
  theme(legend.position="top", plot.title = element_text(hjust = 0.5)) +
  facet_grid(celltype ~ .)

mn_distribution <- ggarrange(mnHistSep, mnHist, common.legend = TRUE, legend = "top")
mn_distribution <- annotate_figure(mn_distribution, top = text_grob("Motor Neuron Distributions along PC1", face = "bold", size = 16))

png("mn_distribution.png", units="in", width=8, height=6, res=300)
mn_distribution
dev.off()


### Figure 2

In [ ]:
# Figures 2a, 2b, 2c made with Biorender.com and Google Slides

# Figure 2d
gp_mtx <- alpha@assays$RNA@data[rownames(alpha@assays$RNA@data) %in% pseudotime_t1000$gene_short_name,]
gp_mtx <- gp_mtx[,names(pseudotime_ordered)] # Reorder

fun <- function(x) {
  rollapply(x, width=25, by=3, FUN="mean") # Scaled row expression: average expression for 10 cells, every 3
}

gp_plot <- Heatmap(t(scale(apply(gp_mtx, 1, fun))), 
            col = colorRampPalette(c("blue", "light yellow", "red"))(n = 100),
            name = "Gene Expression over Pseudotime",
            row_title = "Top 1000 genes",
            column_title = "Fast Score",
            cluster_rows = TRUE,
            clustering_method_rows = "ward.D2",
            show_row_names = FALSE,
            cluster_columns = FALSE,
            show_column_names = FALSE,
            show_heatmap_legend = TRUE,
            row_names_gp = gpar(fontsize = 5))

png("gp_heatmap_1000.png", units="in", width=10, height=8, res=300)
gp_plot
dev.off()

# Figure 2e made with EnrichR web analysis

# Figure 2f
inp <- c("NRP2;SHC3;EPHA10;SEMA3D;SEMA3A;LRTM1;PIK3CB;EFNA5;ROBO1;KIF5C;TNR;NRCAM;EPHB1;NEFH;SPTAN1;NTNG1;NTNG2;EPHA6;DSCAM;DCC;DAB1;MAP1B;PARD3;CNTN4;FGFR2")
g <- capitalize( strsplit(tolower(inp), split=";")[[1]] )
ps <- plot_genes_in_pseudotime(alphaM[g], nrow = 4, ncol = 7, color_cells_by="celltype", min_expr=0.1) + scale_color_manual(values = c('#30DC40','#FC5454','gray'))
png("axonogenesis_pseudotime.png", units="in", width=18, height=6, res=300)
ps
dev.off()

# Figure 2g
g <- c('Cacnb2','Stac','Cacna2d1','Stac2','Nipsnap2')
ps <- plot_genes_in_pseudotime(alphaM[g], nrow = 3, ncol = 2, color_cells_by="celltype", min_expr=0.1) + scale_color_manual(values = c('#30DC40','#FC5454','gray'))
png("poscalc_pseudotime.png", units="in", width=5, height=5, res=300)
ps
dev.off()

### Figure 3

In [ ]:
# Figure 3a made with Biorender.com

# Figure 3b
regulons_df <- read.csv("~/Desktop/mn_paper/f-tables/regulon_summary_df.csv")

EnhanVolc2 <- edit(EnhancedVolcano) # Remove -log10(y) from original function
volcano <- EnhanVolc2(regulons_df,
            lab = regulons_df$TF,
            selectLab = regulons_df$TF[1:20],
            x = "NES.Score",
            y = "NumTargets",
            xlim = c(min(regulons_df$NES.Score, na.rm = TRUE), max(regulons_df$NES.Score, na.rm = TRUE) + 0.75),
            ylim = c(min(regulons_df$NumTargets, na.rm = TRUE) - 1.5, max(regulons_df$NumTargets, na.rm = TRUE) + 1.5),
            xlab = "NES Score",
            ylab = "NumTargets",
            legendLabels = c("Not-selected", "Selected", "Selected", "Selected"),
            pCutoff = 0.00001,
            FCcutoff = 2.0,
            cutoffLineWidth = 0.25,
            pointSize = 3.0,
            labSize = 3.0,
            colAlpha = 0.8,
            legendPosition = 'right',
            title = "Motor Neuron Transcription Factors",
            subtitle = "Divided by confidence (NES) score and target number")
png("tf_volcano.png", units="in", width=10, height=8, res=300)
volcano
dev.off()

# Figure 3c made with Biorender.com

# Figure 3d is a screenshot of table in tf_inference.ipynb

# Figure 3e made in figures_py.ipynb

# Figure 3f made in Cytoscape

# Figure 3g
tfs <- c('Zmiz1','Meis2','Nfib','Meis1','Ikzf2','Ar','Phf21a','Cux2','Maf','Thrb','Pou3f2','Zfp536','Sema4a','Nfia','Arnt2','Polr3g','Sox8')
# Meis1, Nfib, Zfp536 are duplicates
ff_tf <- c('Nfib',
           # 'Nfib',
          'Phf21a',
          'Maf',
          'Thrb',
          'Nfia',
          'Polr3g')
sf_tf <- c('Zmiz1',
          'Meis2',
          # 'Nfib',
          'Meis1',
          'Ikzf2',
          'Ar',
          # 'Meis1',
          'Cux2',
          'Pou3f2',
          'Zfp536',
          'Sema4a',
          # 'Zfp536',
          'Arnt2',
          'Sox8')

ff_tf_pseudotime_plot <- plot_genes_in_pseudotime(alphaM[ff_tf], nrow = 2, ncol = 3, color_cells_by="celltype", min_expr=0.1) +
                        scale_color_manual(values = c('#30DC40','#FC5454','gray'))
png("ff_tf_pseudotime_plot.png", units="in", width=8, height=6, res=300)
ff_tf_pseudotime_plot
dev.off()

# Figure 3h
sf_tf_pseudotime_plot <- plot_genes_in_pseudotime(alphaM[sf_tf], nrow = 3, ncol = 4, color_cells_by="celltype", min_expr=0.1) +
  scale_color_manual(values = c('#30DC40','#FC5454','gray'))
png("sf_tf_pseudotime_plot.png", units="in", width=8, height=6, res=300)
sf_tf_pseudotime_plot
dev.off()


### Figure 4

In [ ]:
list <- c("Zmiz1(-)", "Meis1(+)", "Meis2(-)", "Nfib(-)", "Nfib(+)")

zmiz_target <- read.table("~/Documents/R/gradientWeighing/pyscenic_5/regulons/Zmiz1(-).txt",sep = "\t", header=FALSE)
meis1_target <- read.table("~/Documents/R/gradientWeighing/pyscenic_5/regulons/Meis1(+).txt",sep = "\t", header=FALSE)
meis1_target_2 <- read.table("~/Documents/R/gradientWeighing/pyscenic_5/regulons/Meis1(-).txt",sep = "\t", header=FALSE)
meis1_combined <- rbind(meis1_target, meis1_target_2)
meis2_target <- read.table("~/Documents/R/gradientWeighing/pyscenic_5/regulons/Meis2(-).txt",sep = "\t", header=FALSE)
nfib_n_target <- read.table("~/Documents/R/gradientWeighing/pyscenic_5/regulons/Nfib(-).txt",sep = "\t", header=FALSE)
nfib_p_target <- read.table("~/Documents/R/gradientWeighing/pyscenic_5/regulons/Nfib(+).txt",sep = "\t", header=FALSE)
nfib_combined <- rbind(nfib_n_target, nfib_p_target)

fun <- function(x) {
  rollapply(x, width=25, by=3, FUN="mean") # Scaled row expression: average expression for 10 cells, every 3
}

zmiz_mtx <- alpha@assays$RNA@data[rownames(alpha@assays$RNA@data) %in% zmiz_target$V1,]
zmiz_mtx <- zmiz_mtx[,names(pseudotime_ordered)] # Reorder
gp_plot <- Heatmap(t(scale(apply(zmiz_mtx, 1, fun))), 
                   col = colorRampPalette(c("blue", "light yellow", "red"))(n = 100),
                   name = "Gene Expression over Pseudotime",
                   row_title = "146 Target Genes",
                   column_title = "Fast Score",
                   cluster_rows = TRUE,
                   clustering_method_rows = "ward.D2",
                   show_row_names = TRUE,
                   cluster_columns = FALSE,
                   show_column_names = FALSE,
                   show_heatmap_legend = FALSE,
                   row_names_gp = gpar(fontsize = 5))

meis1_mtx <- alpha@assays$RNA@data[rownames(alpha@assays$RNA@data) %in% meis1_combined$V1,]
meis1_mtx <- meis1_mtx[,names(pseudotime_ordered)] # Reorder
gp_plot <- Heatmap(t(scale(apply(meis1_mtx, 1, fun))), 
                   col = colorRampPalette(c("blue", "light yellow", "red"))(n = 100),
                   name = "Gene Expression over Pseudotime",
                   row_title = "Combined 82 Target Genes",
                   column_title = "Fast Score",
                   cluster_rows = TRUE,
                   clustering_method_rows = "ward.D2",
                   show_row_names = TRUE,
                   cluster_columns = FALSE,
                   show_column_names = FALSE,
                   show_heatmap_legend = FALSE,
                   row_names_gp = gpar(fontsize = 5))

meis2_mtx <- alpha@assays$RNA@data[rownames(alpha@assays$RNA@data) %in% meis2_target$V1,]
meis2_mtx <- meis2_mtx[,names(pseudotime_ordered)] # Reorder
gp_plot <- Heatmap(t(scale(apply(meis2_mtx, 1, fun))), 
                   col = colorRampPalette(c("blue", "light yellow", "red"))(n = 100),
                   name = "Gene Expression over Pseudotime",
                   row_title = "44 Target Genes",
                   column_title = "Fast Score",
                   cluster_rows = TRUE,
                   clustering_method_rows = "ward.D2",
                   show_row_names = TRUE,
                   cluster_columns = FALSE,
                   show_column_names = FALSE,
                   show_heatmap_legend = FALSE,
                   row_names_gp = gpar(fontsize = 5))

nfib_mtx <- alpha@assays$RNA@data[rownames(alpha@assays$RNA@data) %in% c(nfib_combined$V1, "Nfib"),]
nfib_mtx <- nfibp_mtx[,names(pseudotime_ordered)] # Reorder
gp_plot <- Heatmap(t(scale(apply(nfibp_mtx, 1, fun))), 
                   col = colorRampPalette(c("blue", "light yellow", "red"))(n = 100),
                   name = "Gene Expression over Fast Score",
                   row_title = "Combined 30 Target Genes",
                   column_title = "Fast Score",
                   cluster_rows = TRUE,
                   clustering_method_rows = "ward.D2",
                   show_row_names = TRUE,
                   cluster_columns = FALSE,
                   show_column_names = FALSE,
                   show_heatmap_legend = TRUE,
                   row_names_gp = gpar(fontsize = 5))


# Reuse for all four regulons
png("zmiz1_heatmap.png", units="in", width=10, height=8, res=300)
gp_plot
dev.off()

ps_plot <- plot_genes_in_pseudotime(alphaM["Zmiz1"],color_cells_by="celltype", min_expr=0.1) + scale_color_manual(values = c('#30DC40','#FC5454','gray'))
png("zmiz1_pseudotime.png", units="in", width=8, height=6, res=300)
ps_plot
dev.off()

# Gene ontology enrichments done in EnrichR

